In [1]:
!pip install onnx

     |████████████████████████████████| 12.8 MB 5.2 MB/s 


In [2]:
!pip install mlprodict

     |████████████████████████████████| 24.8 MB 1.4 MB/s 
     |████████████████████████████████| 213 kB 55.2 MB/s 


In [3]:
!pip install graphviz

In [4]:
!git clone https://github.com/davidhoof/master-thesis-automatic-selection.git

Cloning into 'master-thesis-automatic-selection'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 119 (delta 46), reused 106 (delta 33), pack-reused 0
Receiving objects: 100% (119/119), 46.40 MiB | 32.37 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [5]:
!git clone https://github.com/onnx/models.git

Cloning into 'models'...
remote: Enumerating objects: 2180, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 2180 (delta 107), reused 234 (delta 43), pack-reused 1820
Receiving objects: 100% (2180/2180), 411.00 MiB | 20.86 MiB/s, done.
Resolving deltas: 100% (800/800), done.
Checking out files: 100% (450/450), done.


In [6]:
!sudo apt-get install git-lfs
!git-lfs install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,247 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 156210 files and directories c

In [7]:
downloadAll=True

In [8]:
import glob
if downloadAll:
  %cd /content/models
  !git lfs pull --include="*" --exclude=""  
  paths = glob.glob('/content/models/vision/classification/**/*.onnx', recursive=True)
else:
  with open("/content/master-thesis-automatic-selection/onnx/test-model-paths.txt") as file:
    paths = [line.rstrip() for line in file]
  for path in paths: 
    model_path=path+".onnx"
    data_path=path+".tar.gz"
    %cd /content/models 
    !git lfs pull --include="$model_path" --exclude=""
    !git lfs pull --include="$data_path" --exclude=""
    

/content/models
Git LFS: (292 of 292 files) 42.43 GB / 42.43 GB


In [9]:
import onnx

In [10]:
import os

#path = "/content/models/vision/classification/mnist/model/mnist-7.onnx"
# path = "/content/models/vision/classification/mobilenet/model/mobilenetv2-7.onnx"
path = "/content/models/vision/classification/resnet/model/resnet152-v2-7.onnx"

model_name = os.path.basename(path)[:-5]
print(model_name)
print(os.path.exists(path))

resnet152-v2-7
True


In [11]:
# path = [s for s in paths if "ssd-10" in s]

In [12]:
model = onnx.load(path)

In [13]:
from mlprodict.onnxrt import OnnxInference

sess = OnnxInference(model)
dot = sess.to_dot()

In [14]:
import graphviz
s = graphviz.Source(dot, filename=model_name,format="pdf")
s.save(directory="/content")
s.view()
s

In [15]:
import sys
sys.path.insert(0,"/content/master-thesis-automatic-selection/searchengine")
from model_resolution.model_resolution import ModelResolution

In [16]:
onnx_model = onnx.load(path)

In [17]:
%time
model_res=ModelResolution(onnx_model)
model_res.minimal_resolution

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


[33, 33]

In [18]:
for node in model_res.filter_nodes(model_res.longest_path):
  print("######################################")
  print(node.name)

######################################
resnetv27_conv0_fwd
######################################
resnetv27_pool0_fwd
######################################
resnetv27_stage1_conv0_fwd
######################################
resnetv27_stage1_conv1_fwd
######################################
resnetv27_stage1_conv2_fwd
######################################
resnetv27_stage1_conv4_fwd
######################################
resnetv27_stage1_conv5_fwd
######################################
resnetv27_stage1_conv6_fwd
######################################
resnetv27_stage1_conv7_fwd
######################################
resnetv27_stage1_conv8_fwd
######################################
resnetv27_stage1_conv9_fwd
######################################
resnetv27_stage2_conv0_fwd
######################################
resnetv27_stage2_conv1_fwd
######################################
resnetv27_stage2_conv2_fwd
######################################
resnetv27_stage2_conv4_fwd
########################

In [19]:
%time
min_resolution=list()
for path in paths:
  if not downloadAll:
    path = "/content/models"+path+".onnx"
  print(path)
  res_m = ModelResolution(path)
  min_resolution.append((os.path.basename(path)[:-5],res_m.minimal_resolution))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs
/content/models/vision/classification/squeezenet/model/squeezenet1.0-6.onnx
/content/models/vision/classification/squeezenet/model/squeezenet1.0-12-int8.onnx
/content/models/vision/classification/squeezenet/model/squeezenet1.0-3.onnx
/content/models/vision/classification/squeezenet/model/squeezenet1.1-7.onnx
/content/models/vision/classification/squeezenet/model/squeezenet1.0-12.onnx
/content/models/vision/classification/squeezenet/model/squeezenet1.0-8.onnx
/content/models/vision/classification/squeezenet/model/squeezenet1.0-7.onnx
/content/models/vision/classification/squeezenet/model/squeezenet1.0-9.onnx
/content/models/vision/classification/inception_and_googlenet/inception_v2/model/inception-v2-6.onnx
/content/models/vision/classification/inception_and_googlenet/inception_v2/model/inception-v2-7.onnx
/content/models/vision/classification/inception_and_googlenet/inception_v2/model/inception-v2-8.onnx
/content/models/vi

In [20]:
min_resolution

[('squeezenet1.0-6', [47, 47]),
 ('squeezenet1.0-12-int8', [47, 47]),
 ('squeezenet1.0-3', [47, 47]),
 ('squeezenet1.1-7', [431, 431]),
 ('squeezenet1.0-12', [47, 47]),
 ('squeezenet1.0-8', [47, 47]),
 ('squeezenet1.0-7', [47, 47]),
 ('squeezenet1.0-9', [47, 47]),
 ('inception-v2-6', [237, 225]),
 ('inception-v2-7', [237, 225]),
 ('inception-v2-8', [237, 225]),
 ('inception-v2-3', [237, 225]),
 ('inception-v2-9', [237, 225]),
 ('googlenet-7', [285, 165]),
 ('googlenet-8', [285, 165]),
 ('googlenet-3', [285, 165]),
 ('googlenet-12-int8', [285, 165]),
 ('googlenet-9', [285, 165]),
 ('googlenet-6', [285, 165]),
 ('googlenet-12', [285, 165]),
 ('inception-v1-12-int8', [93, 93]),
 ('inception-v1-3', [285, 221]),
 ('inception-v1-9', [285, 221]),
 ('inception-v1-7', [285, 221]),
 ('inception-v1-6', [285, 221]),
 ('inception-v1-8', [285, 221]),
 ('inception-v1-12', [285, 221]),
 ('efficientnet-lite4-11-int8', [95, 33]),
 ('efficientnet-lite4-11', [287, 225]),
 ('resnet101-v2-7', [33, 33]),
 ('

In [21]:
%cd /content/
with open('minimal-resolution.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in min_resolution))

/content


In [22]:
for node in onnx.load("/content/models/vision/classification/alexnet/model/bvlcalexnet-12.onnx").graph.node:
  for attribute in node.attribute:
    if(attribute.name=="kernel_shape"):
      print(attribute.ints)
  print(node)

[11, 11]
input: "data_0"
input: "conv1_w_0"
input: "conv1_b_0"
output: "conv1_1"
name: ""
op_type: "Conv"
attribute {
  name: "strides"
  ints: 4
  ints: 4
  type: INTS
}
attribute {
  name: "pads"
  ints: 0
  ints: 0
  ints: 0
  ints: 0
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 11
  ints: 11
  type: INTS
}

input: "conv1_1"
output: "conv1_2"
name: ""
op_type: "Relu"

input: "conv1_2"
output: "norm1_1"
name: ""
op_type: "LRN"
attribute {
  name: "size"
  i: 5
  type: INT
}
attribute {
  name: "alpha"
  f: 9.999999747378752e-05
  type: FLOAT
}
attribute {
  name: "beta"
  f: 0.75
  type: FLOAT
}
attribute {
  name: "bias"
  f: 1.0
  type: FLOAT
}

[3, 3]
input: "norm1_1"
output: "pool1_1"
name: ""
op_type: "MaxPool"
attribute {
  name: "strides"
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "pads"
  ints: 0
  ints: 0
  ints: 0
  ints: 0
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 3
  ints: 3
  type: INTS
}

[5, 5]
input: "pool1_1"
input: "conv2_w_0

In [23]:
!pip install onnxruntime

     |████████████████████████████████| 5.2 MB 5.1 MB/s 


In [24]:
!pip install torch

In [25]:
!python3 -m pip install onnx_graphsurgeon --index-url https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.ngc.nvidia.com


In [26]:
import onnxruntime
import numpy as np
import PIL.Image as Image
import argparse
import onnx_graphsurgeon as gs

def process_image(img_path,input_shape):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(input_shape)
    image = np.array(img, dtype=np.float32)
    image = image.transpose((2,0,1))[np.newaxis, ...]
    return image

img_path = "/content/Koala.jpg"
onnx_path = "/content/resnet152-v2-7.onnx"
print("image path:",img_path)
print("onnx model path:",onnx_path)

image path: /content/Koala.jpg
onnx model path: /content/resnet152-v2-7.onnx


In [27]:
!cp "/content/models/vision/classification/resnet/model/resnet152-v2-7.onnx" "$onnx_path"

In [28]:
graph = gs.import_onnx(onnx.load(onnx_path))

def remove_node(graph,node_name):
  for i in graph.nodes:
    print(i.name)

  remove_node = [
      node for node in graph.nodes if node.name == node_name
  ][0]

  # Get the input node of the fake node
  # Node provides i() and o() functions that can optionally
  # be provided an index (default is 0)
  # These serve as convenience functions for the alternative,
  # which would be to fetch the input/output
  # tensor first, then fetch the input/output node of the tensor.
  # For example, node.i() is equivalent to node.inputs[0].inputs[0]
  inp_node = remove_node.i()

  # Reconnect the input node to the output tensors of the fake node,
  # so that the first identity node in the example graph now
  # skips over the fake node.
  inp_node.outputs = remove_node.outputs
  remove_node.outputs.clear()

  # Remove the fake node from the graph completely
  graph.cleanup()

  h = graph.inputs[0].shape[2]
  w = graph.inputs[0].shape[3]

  scale = 0
  if graph.inputs[0].shape[1] == 4:
      scale = 2
  else:
      scale = 3

  graph.outputs[0].shape = [1,3,h*scale,w*scale]
  return graph

graph=remove_node(graph,"resnetv27_dense0_fwd")
graph=remove_node(graph,"resnetv27_flatten0_reshape0")
# graph=remove_node(graph,"resnetv27_pool1_fwd")
# graph=remove_node(graph,"resnetv27_relu1_fwd")
# graph=remove_node(graph,"resnetv27_batchnorm2_fwd")

# print(graph.outputs)

onnx.save(gs.export_onnx(graph), onnx_path)
print("")

resnetv27_batchnorm0_fwd
resnetv27_conv0_fwd
resnetv27_batchnorm1_fwd
resnetv27_relu0_fwd
resnetv27_pool0_fwd
resnetv27_stage1_batchnorm0_fwd
resnetv27_stage1_activation0
resnetv27_stage1_conv0_fwd
resnetv27_stage1_batchnorm1_fwd
resnetv27_stage1_activation1
resnetv27_stage1_conv1_fwd
resnetv27_stage1_batchnorm2_fwd
resnetv27_stage1_activation2
resnetv27_stage1_conv2_fwd
resnetv27_stage1_conv3_fwd
resnetv27_stage1__plus0
resnetv27_stage1_batchnorm3_fwd
resnetv27_stage1_activation3
resnetv27_stage1_conv4_fwd
resnetv27_stage1_batchnorm4_fwd
resnetv27_stage1_activation4
resnetv27_stage1_conv5_fwd
resnetv27_stage1_batchnorm5_fwd
resnetv27_stage1_activation5
resnetv27_stage1_conv6_fwd
resnetv27_stage1__plus1
resnetv27_stage1_batchnorm6_fwd
resnetv27_stage1_activation6
resnetv27_stage1_conv7_fwd
resnetv27_stage1_batchnorm7_fwd
resnetv27_stage1_activation7
resnetv27_stage1_conv8_fwd
resnetv27_stage1_batchnorm8_fwd
resnetv27_stage1_activation8
resnetv27_stage1_conv9_fwd
resnetv27_stage1__plus2

In [29]:
for node in onnx.load(onnx_path).graph.node:
  print(node.name)

resnetv27_batchnorm0_fwd
resnetv27_conv0_fwd
resnetv27_batchnorm1_fwd
resnetv27_relu0_fwd
resnetv27_pool0_fwd
resnetv27_stage1_batchnorm0_fwd
resnetv27_stage1_activation0
resnetv27_stage1_conv0_fwd
resnetv27_stage1_batchnorm1_fwd
resnetv27_stage1_activation1
resnetv27_stage1_conv1_fwd
resnetv27_stage1_batchnorm2_fwd
resnetv27_stage1_activation2
resnetv27_stage1_conv2_fwd
resnetv27_stage1_conv3_fwd
resnetv27_stage1__plus0
resnetv27_stage1_batchnorm3_fwd
resnetv27_stage1_activation3
resnetv27_stage1_conv4_fwd
resnetv27_stage1_batchnorm4_fwd
resnetv27_stage1_activation4
resnetv27_stage1_conv5_fwd
resnetv27_stage1_batchnorm5_fwd
resnetv27_stage1_activation5
resnetv27_stage1_conv6_fwd
resnetv27_stage1__plus1
resnetv27_stage1_batchnorm6_fwd
resnetv27_stage1_activation6
resnetv27_stage1_conv7_fwd
resnetv27_stage1_batchnorm7_fwd
resnetv27_stage1_activation7
resnetv27_stage1_conv8_fwd
resnetv27_stage1_batchnorm8_fwd
resnetv27_stage1_activation8
resnetv27_stage1_conv9_fwd
resnetv27_stage1__plus2

In [30]:
data_input = process_image(img_path,(224,224))
# session = OnnxInference(onnx_model)
session = onnxruntime.InferenceSession(onnx_path)
# print(session.inputs_)
# print(session.outputs_)

print([output.shape for output in session.get_outputs()])

inname = [input.name for input in session.get_inputs()]
outname = [output.name for output in session.get_outputs()]

print(outname)

# print(inname)
# print(outname)

print("inputs name:",inname,"|| outputs name:",outname)
data_output = session.run(["resnetv27_dense0_fwd"], {inname[0]: data_input})

output = data_output[0][0]
print(output) 

FileNotFoundError: ignored

In [ ]:
import onnxruntime as ort
import numpy as np
import torch
ort_sess = ort.InferenceSession(path)
outputs = ort_sess.run(None, {'input': torch.randn((1, 3, 224, 224)).numpy()})
# Print Result
outputs